# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [54]:
# what to include (continous)
#data = data[['C1', 'C2', 'C4', 'C5', 'C6', 'T1', 'T2', 'T3', 'T4', 'T5', 'C3_impute', 'C7_impute', 'C3`',
       #'C5`', 'C6`', 'T1prime', 'T3prime', 'T4prime', 'S1prime',
       #'S2prime', 'S3prime','Y1', 'Y2',]]

In [2]:
data = pd.read_csv('all_data_y1y2_seperated.csv', header=0)
data = data[['C2','C3`','C5','C3_impute','S3prime','Y1','C2', 'C3`','C4','C5','S3prime','Y2']]
data.head()

,C2,C3`,C5,C3_impute,S3prime,Y1,C2,C3`,C4,C5,S3prime,Y2
0,1,1,-0.085350,0.021250,-0.207260,0,1,1,0.654566,-0.085350,-0.207260,1
1,0,0,-0.200020,-0.010657,-0.225859,1,0,0,-0.619777,-0.200020,-0.225859,0
2,1,0,-0.314033,-0.008256,-0.040877,1,1,0,0.403480,-0.314033,-0.040877,0
3,1,0,-0.332970,-0.006171,0.631816,1,1,0,0.382698,-0.332970,0.631816,0
4,1,1,-0.182264,0.000021,-0.090566,0,1,1,-1.266863,-0.182264,-0.090566,1


In [3]:
data.columns

Index(['C2', 'C3`', 'C5', 'C3_impute', 'S3prime', 'Y1', 'C2', 'C3`', 'C4',
       'C5', 'S3prime', 'Y2'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 12 columns):
C2           660 non-null int64
C3`          660 non-null int64
C5           660 non-null float64
C3_impute    660 non-null float64
S3prime      660 non-null float64
Y1           660 non-null int64
C2           660 non-null int64
C3`          660 non-null int64
C4           660 non-null float64
C5           660 non-null float64
S3prime      660 non-null float64
Y2           660 non-null int64
dtypes: float64(6), int64(6)
memory usage: 62.0 KB


In [5]:
data.isna().sum()

C2           0
C3`          0
C5           0
C3_impute    0
S3prime      0
Y1           0
C2           0
C3`          0
C4           0
C5           0
S3prime      0
Y2           0
dtype: int64

Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [6]:
data.dtypes

C2             int64
C3`            int64
C5           float64
C3_impute    float64
S3prime      float64
Y1             int64
C2             int64
C3`            int64
C4           float64
C5           float64
S3prime      float64
Y2             int64
dtype: object

Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [7]:
y1 = data.Y1
y2 = data.Y2

Check the shape of the data.

In [8]:
data.shape

(660, 12)

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [9]:
data.Y1.value_counts()

1    330
0    330
Name: Y1, dtype: int64

In [10]:
cols = list(data.columns)
# first 5 features 
# -3
cols[:-2]



['C2', 'C3`', 'C5', 'C3_impute', 'S3prime', 'Y1', 'C2', 'C3`', 'C4', 'C5']

Use below code to select the first 5 features as predictors for `Y1`.

In [11]:
# -3
predictors_y1 = data[cols[:-6]] 
#predictors_y1 = data[cols[-2]]#-2
predictors_y1.head()

,C2,C2,C3`,C3`,C5,C5,C3_impute,S3prime,S3prime,Y1
0,1,1,1,1,-0.085350,-0.085350,0.021250,-0.207260,-0.207260,0
1,0,0,0,0,-0.200020,-0.200020,-0.010657,-0.225859,-0.225859,1
2,1,1,0,0,-0.314033,-0.314033,-0.008256,-0.040877,-0.040877,1
3,1,1,0,0,-0.332970,-0.332970,-0.006171,0.631816,0.631816,1
4,1,1,1,1,-0.182264,-0.182264,0.000021,-0.090566,-0.090566,0


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [12]:
# predictors_y2 = data[cols[-2]] #-2
predictors_y2 = data[cols[7:]]
predictors_y2.head()

,C3`,C3`,C4,C5,C5,S3prime,S3prime,Y2
0,1,1,0.654566,-0.085350,-0.085350,-0.207260,-0.207260,1
1,0,0,-0.619777,-0.200020,-0.200020,-0.225859,-0.225859,0
2,0,0,0.403480,-0.314033,-0.314033,-0.040877,-0.040877,0
3,0,0,0.382698,-0.332970,-0.332970,0.631816,0.631816,0
4,1,1,-1.266863,-0.182264,-0.182264,-0.090566,-0.090566,1


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [13]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.2, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is Fal

F1 1.0000; AUC 1.0000 
Accuracy of classifier on test set: 1.00
10-fold cross validation average accuracy of classifier: 1.000
Confusion Matrix for Logistic Regression Classfier:
[[78  0]
 [ 0 54]]
Classification Report for Logistic Regression Classfier:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        78
           1       1.00      1.00      1.00        54

    accuracy                           1.00       132
   macro avg       1.00      1.00      1.00       132
weighted avg       1.00      1.00      1.00       132



Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [14]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))

AttributeError: 'DataFrame' object has no attribute 'value_counts'